# Pulling Tweets from WHO

In [22]:
import GetOldTweets3 as got
import pandas as pd

## Set up Query

In [44]:
username = 'WHO'
count = 3000
since_date = '2020-04-01'
until_date = '2020-06-10'

# Creation of query object
tweetCriteria = got.manager.TweetCriteria().setUsername(username).setSince(since_date).setUntil(until_date).setMaxTweets(count)

# Creation of list that contains all tweets
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

# Create df for tweets
user_tweets = pd.DataFrame([[tweet.date, tweet.text] for tweet in tweets])

In [45]:
user_tweets.shape

(2158, 2)

## Write to CSV

In [48]:
user_tweets.to_csv("../data/april_june_10_2020.csv")